# Mesh Generation Summary (`mesh.edp`)

This notebook summarizes the logic of the FreeFem++ script for generating a mesh with random circular reflectors in a square tissue domain.

---

## **Parameters**

- $c_0$: background speed of sound (default: 1540 m/s)
- $\sigma$: standard deviation for speed of sound in reflectors
- $T$: half-length of the tissue square domain (default: $5.0 \times 10^{-3}$ m)
- $\text{radius}$: radius of each reflector
- $\rho$: target filling ratio of reflectors

The number of reflectors is computed as:
$$
N_\text{reflectors} = \left\lfloor \rho \frac{4T^2}{\pi \cdot \text{radius}^2} \right\rceil
$$

---

## **Reflector Placement**

- Each reflector center $(x_i, y_i)$ is randomly generated within the square, at least $\text{radius} + \delta$ from the edge.
- Reflectors are accepted only if their centers are at least $\text{minDistance} = 2 \times \text{radius} + \delta$ apart (no overlap).
- Only valid (non-overlapping) reflectors are kept.

---

## **Reflector Boundary**

Each reflector is defined parametrically:
$$
x = x_i + \text{radius} \cdot \cos(t), \quad y = y_i + \text{radius} \cdot \sin(t), \quad t \in [0, 2\pi]
$$

---

## **Mesh Construction**

- The tissue boundary is a square of side $2T$.
- The mesh is built from the square boundary and all reflector boundaries.

The final mesh is constructed by combining:

- The four square domain borders (tissue boundary), each discretized using \( n_{2T} \) points. This number is chosen to match the mesh resolution set by the reflectors' boundary mesh size.
- All reflector boundaries, with each being discretized using `nbndR` points.

In **FreeFem++**, the relevant mesh-building code is:

```cpp
mesh Th = buildmesh(
  BTtop(n2T) + BTleft(n2T) + BTbottom(n2T) + BTright(n2T) +
  reflector(reflectornbpts),
  nbvx=10^8
);

---

## **Material Properties**

- Each reflector is assigned a random speed of sound:
  $$
  c_i = c_0 + \mathcal{N}(0, \sigma)
  $$
- The background uses $c_0$.
- The speed of sound function is:
  $$
  c(\text{region}) = 
  \begin{cases}
    c_i & \text{if region $i$ is a reflector} \\
    c_0 & \text{otherwise}
  \end{cases}
  $$

---

For more details, see the FreeFem++ documentation: [https://freefem.org/](https://freefem.org/)